In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("restaurant-review-average")
sc = SparkContext(conf=conf)
sc

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/27 10:23:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local appName=restaurant-review-average>

In [2]:
filepath = "/home/ubuntu/working/spark/data/restaurant_reviews.csv"

In [3]:
!pwd # 데이터 경로

/home/ubuntu/working/spark


In [4]:
!ls data # 데이터 이름

restaurant_reviews.csv	xAPI-Edu-Data.csv


In [5]:
lines = sc.textFile(f"file:///{filepath}")
lines.collect()

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [6]:
# header 제거
header = lines.first()
rows = lines.filter(lambda row : row != header)
rows

PythonRDD[3] at RDD at PythonRDD.scala:53

In [7]:
rows.collect()

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [8]:
# map 을 활용해 각 행을 파싱하기 위한 TASK(함수)
# lambda 는 일회성함수로 여러가지 할려 할 땐 def 함수를 쓰자
def parse(row):
    fields = row.split(",")
    
    # 카테고리
    category = fields[2]
    
    # 리뷰 개수
    review_count = fields[3]
    review_count = int(review_count)
    
    return category, review_count

In [9]:
# 잘 작동하는지 테스트
parse('4,라멘,일식,223')

('일식', 223)

In [10]:
# RDD 내의 모든 원소(row)에 대해 parse 함수를 적용하고 적용된 결과를 추출 -> map
category_reviews = rows.map(parse)
category_reviews

PythonRDD[4] at RDD at PythonRDD.scala:53

In [11]:
category_reviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [18]:
category_reviews_count = category_reviews.mapValues(lambda review_count : (review_count, 1))
category_reviews_count

PythonRDD[9] at RDD at PythonRDD.scala:53

In [19]:
category_reviews_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [20]:
reduce = category_reviews_count.reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))
# lambda 함수 쪽은 TASK 이며, 누적 연산을 할 수 있다
reduce0

PythonRDD[14] at RDD at PythonRDD.scala:53

In [22]:
reduce.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [23]:
average = reduce.mapValues(lambda v : v[0] / v[1])
average

PythonRDD[15] at RDD at PythonRDD.scala:53

In [24]:
average.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

In [25]:
sc.stop()